In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
test = pd.read_csv('test_accounts.csv')
train = pd.read_csv('train.csv')
test.shape, train.shape

((6300, 1), (5826604, 4))

In [3]:
train['value'] = train['value'].astype(float)
train['gas_fee'] = train['gas'] * train['gas_price']
train.head()

,value,gas,gas_price,account,gas_fee
0,0.000000e+00,72585,11500000000,a00996,834727500000000
1,0.000000e+00,54426,11349723260,a07890,617720038148760
2,0.000000e+00,200000,14024584890,a22857,2804916978000000
3,1.089000e+17,21000,11349723260,a07890,238344188460000
4,0.000000e+00,149999,32000000000,a21390,4799968000000000


In [4]:
train = train.groupby('account').mean()
print(train.shape)
print(train.isna().sum())
train.head()

(31491, 4)
value        0
gas          0
gas_price    0
gas_fee      0
dtype: int64


,value,gas,gas_price,gas_fee
account,,,,
a00001,1.916964e+16,73358.818182,5.209091e+10,3.308872e+15
a00002,3.724191e+16,100129.666667,7.766667e+10,4.954446e+15
a00003,4.039974e+19,34800.000000,2.100000e+10,7.308000e+14
a00004,1.260368e+17,21210.526316,8.664211e+10,1.842474e+15
a00005,1.101349e+17,27860.465116,1.834833e+10,4.508707e+14


In [5]:
tf = pd.merge(train, test, on='account', how='right')

In [6]:
print(tf.shape)
tf.head()

(6300, 5)


,account,value,gas,gas_price,gas_fee
0,a27890,9.361828e+13,86951.111869,3.050435e+09,2.602325e+14
1,a29649,7.206806e+12,131479.870246,2.850131e+10,4.119353e+15
2,a28243,4.967957e+10,82650.023424,3.799208e+10,3.067196e+15
3,a07155,7.471264e+15,428157.039715,9.316217e+10,3.686012e+16
4,a15576,1.003361e+15,115034.567974,7.041525e+09,8.160464e+14


In [7]:
tf.isna().sum()

account      0
value        2
gas          2
gas_price    2
gas_fee      2
dtype: int64

In [8]:
# fill missing values with mean
tf['value'] = tf['value'].fillna(tf['value'].mean())
tf['gas'] = tf['gas'].fillna(tf['gas'].mean())
tf['gas_price'] = tf['gas_price'].fillna(tf['gas_price'].mean())
tf['gas_fee'] = tf['gas_fee'].fillna(tf['gas_fee'].mean())
tf.isna().sum()

account      0
value        0
gas          0
gas_price    0
gas_fee      0
dtype: int64

In [9]:
x = tf.drop(['account'], axis=1)
x.head()

,value,gas,gas_price,gas_fee
0,9.361828e+13,86951.111869,3.050435e+09,2.602325e+14
1,7.206806e+12,131479.870246,2.850131e+10,4.119353e+15
2,4.967957e+10,82650.023424,3.799208e+10,3.067196e+15
3,7.471264e+15,428157.039715,9.316217e+10,3.686012e+16
4,1.003361e+15,115034.567974,7.041525e+09,8.160464e+14


In [10]:
x.isna().sum()

value        0
gas          0
gas_price    0
gas_fee      0
dtype: int64

In [11]:
# load the model from disk
import pickle
knn1 = pickle.load(open('knn1.pkl', 'rb'))

In [12]:
predictions = knn1.predict(x)
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
predictions = pd.DataFrame(predictions, columns=['flag'])
predictions.head()

,flag
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [14]:
res = pd.concat([test, predictions], axis=1)
res.head()

,account,flag
0,a27890,0.0
1,a29649,0.0
2,a28243,0.0
3,a07155,0.0
4,a15576,0.0


In [17]:
# convert flag to int
res['flag'] = res['flag'].astype(int)
res.head()

,account,flag
0,a27890,0
1,a29649,0
2,a28243,0
3,a07155,0
4,a15576,0


In [18]:
res['flag'].value_counts()

0    3916
1    2384
Name: flag, dtype: int64

In [19]:
res.to_csv('submission.csv', index=False)